In [1]:
import numpy as np
import pandas as pd
import keras as k
from keras.layers import Merge
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from keras.callbacks import History
from keras.layers import Activation
from keras.models import model_from_json
from keras.optimizers import Adam
from matplotlib import pyplot as plt
from scipy.ndimage import rotate as rot
np.random.seed(100)

Using TensorFlow backend.


In [4]:
file_path = '../data/train.json'
train = pd.read_json(file_path)

print(train.head())
train.shape

                                              band_1  \
0  [-27.878360999999998, -27.15416, -28.668615, -...   
1  [-12.242375, -14.920304999999999, -14.920363, ...   
2  [-24.603676, -24.603714, -24.871029, -23.15277...   
3  [-22.454607, -23.082819, -23.998013, -23.99805...   
4  [-26.006956, -23.164886, -23.164886, -26.89116...   

                                              band_2        id inc_angle  \
0  [-27.154118, -29.537888, -31.0306, -32.190483,...  dfd5f913   43.9239   
1  [-31.506321, -27.984554, -26.645678, -23.76760...  e25388fd   38.1562   
2  [-24.870956, -24.092632, -20.653963, -19.41104...  58b2aaa0   45.2859   
3  [-27.889421, -27.519794, -27.165262, -29.10350...  4cfc3a18   43.8306   
4  [-27.206915, -30.259186, -30.259186, -23.16495...  271f93f4   35.6256   

   is_iceberg  
0           0  
1           0  
2           1  
3           0  
4           0  


(1604, 5)

In [5]:
train[train['inc_angle'] == 'na'].count()
train.inc_angle = train.inc_angle.map(lambda x: 0.0 if x == 'na' else x)


In [6]:
def transform (df):
    images = []
    for i, row in df.iterrows():
        band_1 = np.array(row['band_1']).reshape(75,75)
        band_2 = np.array(row['band_2']).reshape(75,75)
        band_3 = band_1 + band_2
        
        band_1_norm = (band_1 - band_1.mean()) / (band_1.max() - band_1.min())
        band_2_norm = (band_2 - band_2. mean()) / (band_2.max() - band_2.min())
        band_3_norm = (band_3 - band_3.mean()) / (band_3.max() - band_3.min())
        
        images.append(np.dstack((band_1_norm, band_2_norm, band_3_norm)))
    
    return np.array(images)

In [11]:
def augment(images):
    image_mirror_lr = []
    image_mirror_ud = []
    image_rotate = []
    for i in range(0,images.shape[0]):
        band_1 = images[i,:,:,0]
        band_2 = images[i,:,:,1]
        band_3 = images[i,:,:,2]
            
        # mirror left-right
        band_1_mirror_lr = np.fliplr(band_1)
        band_2_mirror_lr = np.fliplr(band_2)
        band_3_mirror_lr = np.fliplr(band_3)
        image_mirror_lr.append(np.dstack((band_1_mirror_lr, band_2_mirror_lr, band_3_mirror_lr)))
        
        # mirror up-down
        band_1_mirror_ud = np.flipud(band_1)
        band_2_mirror_ud = np.flipud(band_2)
        band_3_mirror_ud = np.flipud(band_3)
        image_mirror_ud.append(np.dstack((band_1_mirror_ud, band_2_mirror_ud, band_3_mirror_ud)))
        
        #rotate 
        band_1_rotate = rot(band_1, 30, reshape=False)
        band_2_rotate = rot(band_2, 30, reshape=False)
        band_3_rotate = rot(band_3, 30, reshape=False)
        image_rotate.append(np.dstack((band_1_rotate, band_2_rotate, band_3_rotate)))
        
    mirrorlr = np.array(image_mirror_lr)
    mirrorud = np.array(image_mirror_ud)
    rotated = np.array(image_rotate)
    images = np.concatenate((images, mirrorlr, mirrorud, rotated))
    return images


In [12]:
train_x = transform(train)
train_y = np.array(train ['is_iceberg'])

indx_tr = np.where(train.inc_angle > 0)
print (indx_tr[0].shape)

train_y = train_y[indx_tr[0]]
train_x = train_x[indx_tr[0], ...]

train_x = augment(train_x)
train_y = np.concatenate((train_y,train_y, train_y, train_y))

print (train_x.shape)
print (train_y.shape)

(1471,)
(5884, 75, 75, 3)
(5884,)


In [13]:
model = k.models.Sequential()

model.add(k.layers.convolutional.Conv2D(64, kernel_size=(3,3), input_shape=(75,75,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(k.layers.convolutional.MaxPooling2D(pool_size=(3,3), strides=(2,2)))
model.add(k.layers.Dropout(0.2))

model.add(k.layers.convolutional.Conv2D(128, kernel_size=(3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(k.layers.convolutional.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(k.layers.Dropout(0.2))

model.add(k.layers.convolutional.Conv2D(128, kernel_size=(3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(k.layers.convolutional.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(k.layers.Dropout(0.3))

model.add(k.layers.convolutional.Conv2D(64, kernel_size=(3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(k.layers.convolutional.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(k.layers.Dropout(0.3))

model.add(k.layers.Flatten())

model.add(k.layers.Dense(512))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(k.layers.Dropout(0.2))

model.add(k.layers.Dense(256))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(k.layers.Dropout(0.2))


model.add(k.layers.Dense(1))
model.add(Activation('sigmoid'))

mypotim=Adam(lr=0.01, decay=0.0)
model.compile(loss='binary_crossentropy', optimizer = mypotim, metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 73, 73, 64)        1792      
_________________________________________________________________
activation_1 (Activation)    (None, 73, 73, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 73, 73, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
activation_2 (Activation)    (None, 34, 34, 128)       0         
__________

In [16]:
batch_size = 64
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 10, verbose = 0, mode= 'min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor = 0.1, patience = 7, verbose =1, 
                                   epsilon = 1e-4, mode='min', min_lr = 0.0001)
model_filepath='weights.best.hdf5'
checkpoint = ModelCheckpoint(model_filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [early_stopping, checkpoint]
history = model.fit(train_x, train_y, batch_size = batch_size, epochs =20, verbose =1, validation_split = 0.1, 
          callbacks=callbacks_list)

Train on 5295 samples, validate on 589 samples
Epoch 1/20
5295/5295 [==============================] - 129s 24ms/step - loss: 0.5159 - acc: 0.7679 - val_loss: 7.9359 - val_acc: 0.5076
Epoch 2/20
5295/5295 [==============================] - 126s 24ms/step - loss: 0.3263 - acc: 0.8510 - val_loss: 7.9359 - val_acc: 0.5076
Epoch 3/20
5295/5295 [==============================] - 126s 24ms/step - loss: 0.2810 - acc: 0.8782 - val_loss: 7.9359 - val_acc: 0.5076
Epoch 4/20
5295/5295 [==============================] - 126s 24ms/step - loss: 0.2678 - acc: 0.8861 - val_loss: 5.4771 - val_acc: 0.5076
Epoch 5/20
5295/5295 [==============================] - 126s 24ms/step - loss: 0.2387 - acc: 0.8963 - val_loss: 2.9593 - val_acc: 0.5076
Epoch 6/20
5295/5295 [==============================] - 126s 24ms/step - loss: 0.2376 - acc: 0.9016 - val_loss: 0.2711 - val_acc: 0.8778
Epoch 7/20
5295/5295 [==============================] - 126s 24ms/step - loss: 0.2217 - acc: 0.9073 - val_loss: 0.9303 - val_acc: 0

In [17]:
print (history.history.keys())
fig = plt.figure()
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','test'],loc='upper left')
plt.show()


dict_keys(['val_loss', 'acc', 'loss', 'val_acc'])


In [19]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [20]:
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights('weights.best.hdf5')
print("Loaded model from disk")
loaded_model.compile(loss='binary_crossentropy', optimizer = mypotim, metrics=['accuracy'])

Loaded model from disk


In [22]:
test_file = '../data/test.json'
test = pd.read_json(test_file)
test.inc_angle = test.inc_angle.replace('na',0)
test_x = transform(test)
print (test_x.shape)

(8424, 75, 75, 3)


In [23]:
pred_test = loaded_model.predict(test_x, verbose=1)
submission = pd.DataFrame({'id': test["id"], 'is_iceberg': pred_test.reshape((pred_test.shape[0]))})
submission.to_csv('submission.csv', index=False)

8424/8424 [==============================] - 80s 9ms/step
